In the program, we have the last digit of sid `7`, so the order no. is `7`,$S1$ is 688 *China Overseas*, initial stock price `12.18`, volatility `43.6%`; $S2$ is 857 *Petrochina*, initial stock price `6.03`, volatility `30.0%`; $S1/S2$ correlation coefficient `0.304`, Group 3.

For Group 3, the $F\%=102.6\%, UB\%=130.0\%, A\%=122.0\%$

### Q3

#### Product description
HSBC's shark fin structed note with floor, based on stock $S1$

| Norminal amount (NOM)           | HK$100,000                                                   |
| ------------------------------- | ------------------------------------------------------------ |
| Start Date ($D_s$)             | $t$                                                          |
| Initial stock price             | $S_0$                                                        |
| Stock price before maturity     | $S$                                                          |
| Expiry Date ($T$)               | $t+1.25$ year                                                |
| Expiry date stock closing price | $S_M$                                                        |
| Strike ($K$)                    | $S_0 * 100\%$                                                |
| Barrier price ($P_u$)           | $S_0 * UB\%$                                                 |
| Floor                           | $F\%$                                                        |
| Participation rate              | $A\%$                                                        |
| Payoff at maturity              | 1. payoff = $NOM*F\%$, if the barrier event is triggered          OR<br />2. payoff = $NOM*[100\%+A*\max(S_M/S_0-100\%, 0)]$, if the barrier event is triggered |
| Barrier event                   | $S>P_u$ at any time before maturity or $S_M>P_u$ at maturity |

Continuously compounded interest rate *r = 4.34% p.a.*

$F\%, UB\%$ and $A\%$ should be based on the Group number of stock *S1*. 

### Q3(i)

Calculate the fair price of this product using a Monte Carlo scheme with time steps $N = 250, i.e. \Delta t = T/N = 1/200$ (refer to the discretization scheme in Topic 1-2, slides 37 and 38).  Give the answer with 100000 paths. 

In [1]:
import numpy as np

# Parameters
S0 = 12.18
sigma = 0.436
r = 0.0434
T = 1.25
N = 250
dt = T / N
num_paths = 100000
UB_pct = 130.0
Pu = S0 * UB_pct / 100
F = 102.6 / 100 
A = 122 / 100 
NOM = 100000

np.random.seed(42)  # For reproducibility
epsilons = np.random.normal(size=(num_paths, N))

# Calculate stock paths
drift = (r - 0.5 * sigma**2) * dt
volatility = sigma * np.sqrt(dt)
increments = drift + volatility * epsilons
cumulative_increments = np.cumsum(increments, axis=1)
log_S = np.log(S0) + cumulative_increments
S = np.exp(log_S)

# Check barrier triggering
triggered = np.any(S >= Pu, axis=1)
S_M = S[:, -1]

# Calculate payoff
payoff = np.where(triggered, NOM * F, NOM * (1 + A * np.maximum(S_M/S0 - 1, 0)))

# Discount to present value
present_value = payoff * np.exp(-r * T)
fair_price = np.mean(present_value)

print(f"Q3(i) Fair Price: HK${fair_price:.2f} ({fair_price / NOM * 100:.2f}% of NOM)")

Q3(i) Fair Price: HK$96673.93 (96.67% of NOM)


### Q3(ii)

If the product is sold to an investor at a price of 100% of the nominal amount, calculate the initial profit of the investment bank. 

In [2]:
100000 - fair_price

3326.0663425905223

### Q3(iii)

The investment bank wants to increase the initial profit to 2.25% of the nominal amount.  If F% is left unchanged, this can be achieved in two ways: (a) lowering the barrier UB% while keeping A% unchanged; (b) lowering the participation rate A%, while keeping UB% unchanged.   By trial and error with 100000 paths, give the answers for (a) and (b), with accuracy up to 0.5% (e.g. a possible answer for UB is 122.0% or 122.5%). 

In [3]:
UB_pct_trial = 116.5
Pu_trial = S0 * UB_pct_trial / 100
triggered_trial = np.any(S >= Pu_trial, axis=1)
payoff_trial = np.where(triggered_trial, NOM * F, NOM * (1 + A * np.maximum(S_M/S0 - 1, 0)))
fair_price_trial = np.mean(payoff_trial * np.exp(-r * T))
print(f"UB% = {UB_pct_trial}%: {fair_price_trial / NOM * 100:.2f}%")

UB% = 116.5%: 96.53%
